# Test A Model on a Random Test Image

In [5]:
%load_ext autoreload
%autoreload 2

import os
import glob
from yaml import safe_load as yaml_load

os.environ['CUDA_VISIBLE_DEVICES'] = ''

import numpy as np
import tifffile
import matplotlib.pyplot as plt

from stardist import random_label_cmap

from keras_transfer_learning import model, dataset, utils
from mean_average_precision import mean_ap

lbl_cmap = random_label_cmap()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## List the models

In the following cell all available models in the model directory are listed. Choose one by copying its name to the next cell.

In [2]:
print('\n'.join(utils.utils.list_model_names()))

P/cityscapes/dsb2018/stardist/resnet-unet/002/000
P/cityscapes/dsb2018/stardist/resnet-unet/002/001
P/cityscapes/dsb2018/stardist/resnet-unet/002/002
P/cityscapes/dsb2018/stardist/resnet-unet/002/003
P/cityscapes/dsb2018/stardist/resnet-unet/002/004
P/cityscapes/dsb2018/stardist/resnet-unet/005/000
P/cityscapes/dsb2018/stardist/resnet-unet/005/001
P/cityscapes/dsb2018/stardist/resnet-unet/005/002
P/cityscapes/dsb2018/stardist/resnet-unet/005/003
P/cityscapes/dsb2018/stardist/resnet-unet/005/004
P/cityscapes/dsb2018/stardist/resnet-unet/010/000
P/cityscapes/dsb2018/stardist/resnet-unet/010/001
P/cityscapes/dsb2018/stardist/resnet-unet/010/002
P/cityscapes/dsb2018/stardist/resnet-unet/010/003
P/cityscapes/dsb2018/stardist/resnet-unet/010/004
P/cityscapes/dsb2018/stardist/resnet-unet/050/000
P/cityscapes/dsb2018/stardist/resnet-unet/050/001
P/cityscapes/dsb2018/stardist/resnet-unet/050/002
P/cityscapes/dsb2018/stardist/resnet-unet/050/003
P/cityscapes/dsb2018/stardist/resnet-unet/050/004


In [3]:
model_name = 'R/none/dsb2018/stardist/unet/F/000'

In [4]:
model_dir = os.path.join('.', 'models', model_name)
m = model.Model(model_dir=model_dir, load_weights='last')

Instructions for updating:
Colocations handled automatically by placer.


## Run The Model on a Random Example

Set the seed to some integer number to get the same example every time.

In [22]:
seed = None # Change to load another example
num = 20
outdir = '/tmp/test_eval'
os.mkdir(outdir)

preddir = os.path.join(outdir, 'pred')
os.mkdir(preddir)
gtdir = os.path.join(outdir, 'gt')
os.mkdir(gtdir)

outfile = os.path.join(outdir, '{:04d}.tif')

d = dataset.Dataset(m.config)

# Get random images
imgs, labels = zip(*[d.get_random_test_img() for i in range(num)])
imgs, labels = list(imgs), list(labels)

# Predict on the images
pred_labels, scores = zip(*m.predict_and_process(imgs))
pred_labels, scores = list(pred_labels), list(scores)

# Save the prediction
for i, pred in enumerate(pred_labels):
    tifffile.imsave(os.path.join(preddir, '{:04d}.tif').format(i), pred)
    
# Save the gt
for i, lab in enumerate(labels):
    tifffile.imsave(os.path.join(gtdir, '{:04d}.tif').format(i), lab)

# Compute the mean AP
ap = mean_ap.ap_dsb2018(pred_labels, labels)

print(ap)

100%|██████████| 20/20 [00:35<00:00,  2.09s/it]


[0.7971272638605251, 0.7546087307784822, 0.7255186663019632, 0.6777523718701739, 0.6200479477298383, 0.5693080762378403, 0.4747676607805798, 0.3191320510416081, 0.14781950521535572, 0.008920081937216842]
